In [1]:
import pickle
import os
import pandas as pd
import numpy as np
from collections import defaultdict
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

In [2]:
# walk the experiment directory
all_experiments = {}
for root, dirs, files in os.walk('../experiments/experiments'):
    experiment = defaultdict(dict)
    for f in files:
        key = '/'.join(root.split('/')[1:-1])    
        trial = root.split('/')[-1]
        if f == "options.pkl":
            opts = vars(pickle.load(open(os.path.join(root, f), 'rb')))
            experiment = all_experiments.get(key, defaultdict(dict))
            experiment[trial]['opts'] = opts
            all_experiments[key] = experiment
        elif f.startswith("event"):
            experiment = all_experiments.get(key, defaultdict(dict))
            experiment[trial]['event_accumulator'] = EventAccumulator(os.path.join(root, f))
            all_experiments[key] = experiment


In [3]:
def filter_experiments(experiments):
    accumulators = []
    for exp in experiments.values():
        if exp.values()[0].get('event_accumulator') is not None:
            accumulators.append(exp)
    return accumulators

In [4]:
experiments = filter_experiments(all_experiments)

In [43]:
def make_trial_df(trial):
    scalar_tags = trial['event_accumulator'].Reload().Tags()['scalars']
    opt_tags = trial['opts'].keys()
    my_tags = ['auc_train', 'auc_valid', 'auc_test', 'acc_train', 'acc_valid', 'acc_test']
    df = pd.DataFrame(columns=opt_tags + my_tags)
    event_accumulator = trial['event_accumulator'].Reload()
    events = event_accumulator.Scalars('auc_valid')
    best_epoch = 3
    scalars = np.array(map(lambda x: x.value, events))
    df.loc[:, tag] = scalars
    for tag in opt_tags:
        df.loc[:, tag] = [trial['opts'][tag]]
    return df

In [ ]:
raw_df = None
for exp in experiments:
    exp_df = None
    for key, trial in exp.iteritems():
        if exp_df is not None:
            exp_df = exp_df.append(make_trial_df(trial))
        else:
            exp_df = make_trial_df(trial)
    if raw_df is not None:   
        raw_df = raw_df.append(exp_df)
    else:
        raw_df = exp_df


In [37]:
def take_mean(events):
    print events
    mean = np.array([e.value() for e in events]).mean()
    import pdb; pdb.set_trace()
    print mean

In [ ]:
e = trial['event_accumulator']
auc = e.Scalars('auc_train')
take_mean(auc)
import pdb; pdb.set_trace()

--Call--
> /Tmp/lisa/os_v5/anaconda/lib/python2.7/site-packages/IPython/core/displayhook.py(236)__call__()
-> def __call__(self, result=None):
(Pdb) auc
*** NameError: name 'auc' is not defined
(Pdb) print auc
*** NameError: name 'auc' is not defined
(Pdb) print e
*** NameError: name 'e' is not defined
(Pdb) q


In [ ]:
trial = experiments.values()[0].values()[0]
scalar_tags = trial['event_accumulator'].Reload().Tags()['scalars']
opt_tags = trial['opts'].keys()
df = pd.DataFrame(columns=opt_tags + scalar_tags)
exp_df = pd.DataFrame(columns=opt_tags + scalar_tags)
for path, experiment in experiments.iteritems():
    for number, trial in experiment.iteritems():
        import pdb; pdb.set_trace()

        event_accumulator = trial['event_accumulator'].Reload()
        for tag in scalar_tags:
            events = event_accumulator.Scalars(tag)
            scalars = np.array(map(lambda x: x.value, events))
            df.loc[:, tag] = scalars
        for tag in opt_tags:
            df.loc[:, tag] = [trial['opts'][tag]]
        exp_df = exp_df.append(df)


In [ ]:
for path, experiment in experiments.iteritems():
    for number, trial in experiment.iteritems():
        print trial